In [79]:
import msprime
import numpy as np
from IPython.display import SVG, display

import concurrent.futures

In [80]:
# demography parameters
dem_size = 1e5
T_split = 1000
split_proportion=0.95
T_p3 = 15

In [6]:
dem = msprime.Demography()
dem.add_population(name="p0", initial_size=dem_size*2)
# two original populations
dem.add_population(name="p1", initial_size=dem_size)
dem.add_population(name="p2", initial_size=dem_size)
# admixture population
dem.add_population(name="p3", initial_size=dem_size*0.5)
# split_proportion is unknown
dem.add_admixture(time=15, derived="p3", ancestral=["p1", "p2"],
                  proportions=[split_proportion, 1-split_proportion])
dem.add_population_split(time=1000, derived=["p1", "p2"], ancestral="p0")

NameError: name 'msprime' is not defined

In [5]:
for pop in dem.populations:
    print(pop.name, pop.id)

p1 0
p2 1
p0 2
p3 3


l3 = l1 * a + l2 * (1-a)

a = (l3 - l2) / (l1 - l2) -> цель

Генерим параллельно!!

## 1. Распределение через долю соседей

In [25]:
def sample_proportion(treeseq, sample):
    occur = 0
    for tree in treeseq.trees():
        siblings = tree.siblings(sample)
        nearest = list([leaf for leaf in tree.leaves(sibling)] for sibling in siblings)[0]
        nearest_pop = [tree.population(nd) for nd in nearest]
        occur += nearest_pop.count(0) * tree.span / len(nearest_pop)
    occur /= treeseq.sequence_length
    return occur

def mean_proportions(ts, samples_set):
    proportions = np.array([sample_proportion(ts, s) for s in samples_set])
    return np.mean(proportions)

In [44]:
%%timeit
treeseq = msprime.sim_ancestry(
        samples={"p1": p_size, "p2": p_size, "p3": p3_size},
        demography=dem,
        sequence_length=3e5, # надо 3e6
        recombination_rate=1e-9, # надо 1e-8
        random_seed=seed,
        # record_migrations=True,    
        # record_provenance=True
)
samples_p1 = treeseq.samples(population_id=0)
samples_p2 = treeseq.samples(population_id=1)
samples_p3 = treeseq.samples(population_id=3)
print(samples_p1.size, samples_p2.size, samples_p3.size, treeseq.num_trees)
L1 = mean_proportions(treeseq, samples_p1)
print("L1 passed")
L2 = mean_proportions(treeseq, samples_p2)
print("L2 passed")
L3 = mean_proportions(treeseq, samples_p3)
print("L3 passed")
[L1, L2, L3]

2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
2000 2000 2000 1078
L1 passed
L2 passed
L3 passed
50.3 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
p_size = 3000
p3_size = 3000 # в перспективе для наглядности нужно больше

In [45]:
def worker(seed):
    treeseq = msprime.sim_ancestry(
        samples={"p1": p_size, "p2": p_size, "p3": p3_size},
        demography=dem,
        sequence_length=3e5, # надо 3e6
        recombination_rate=1e-9, # надо 1e-8
        random_seed=seed,
        # record_migrations=True,    
        # record_provenance=True
    )
    samples_p1 = treeseq.samples(population_id=0)
    samples_p2 = treeseq.samples(population_id=1)
    samples_p3 = treeseq.samples(population_id=3)
    
    L1 = mean_proportions(treeseq, samples_p1)  
    L2 = mean_proportions(treeseq, samples_p2)
    L3 = mean_proportions(treeseq, samples_p3)
    return [L1, L2, L3]

In [47]:
num_replicates = 10
rng = np.random.RandomState(42)
seeds = rng.randint(1, 2**31, size=num_replicates)
seq_length = 3e5
rec_rate = 1e-9

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(worker, seeds)
    S = np.array(list(results))
L1 = np.mean(S[:, 0])
L2 = np.mean(S[:, 1])
L3 = np.mean(S[:, 2])
np.mean(L1), np.mean(L2), np.mean(L3)

(np.float64(0.9222218046038165),
 np.float64(0.0001783248431078886),
 np.float64(0.07745019335917316))

In [85]:
L1, L2, L3

(np.float64(0.9222218046038165),
 np.float64(0.0001783248431078886),
 np.float64(0.07745019335917316))

In [48]:
(L3 - L2)/(L1 - L2)

np.float64(0.08380501593712164)

Эта цифра была получена с $\alpha$ = 0.2...

### Одиночные запуски

In [82]:
treeseq = msprime.sim_ancestry(
        samples={"p1": p_size, "p2": p_size, "p3": p3_size},
        demography=dem,
        sequence_length=3e5, # надо 3e6
        recombination_rate=1e-9, # надо 1e-8
        random_seed=seed,
        # record_migrations=True,    
        # record_provenance=True
    )
samples_p1 = treeseq.samples(population_id=0)
samples_p2 = treeseq.samples(population_id=1)
samples_p3 = treeseq.samples(population_id=3)

l1 = mean_proportions(treeseq, samples_p1)  
l2 = mean_proportions(treeseq, samples_p2)
l3 = mean_proportions(treeseq, samples_p3)

In [83]:
(l3-l2)/(l1 - l2)

np.float64(0.3010852163637988)

### 1.5 На одного соседа повыше

## Распределение через средневзвешенных соседей

In [1]:
def sample_neighbours_weights(treeseq, sample):
    occur1, occur2 = 0, 0 # occur3 тоже считать??
    for tree in treeseq.trees():
        siblings = tree.siblings(sample)
        nearest = list([leaf for leaf in tree.leaves(sibling)] for sibling in siblings)[0]
        nearest_pop = [tree.population(nd) for nd in nearest]
        occur1 += nearest_pop.count(0) * tree.span #/ len(nearest_pop)
        occur2 += nearest_pop.count(1) * tree.span
    # occur /= treeseq.sequence_length
    return occur1, occur2

def mean_weight_proportions(ts, samples_set):
    proportions = np.array([sample_neighbours_weights(ts, s) for s in samples_set]) / ts.sequence_length
    return np.mean(proportions[:,0]), np.mean(proportions[:,1])

In [88]:
treeseq = msprime.sim_ancestry(
        samples={"p1": p_size, "p2": p_size, "p3": p3_size},
        demography=dem,
        sequence_length=3e5, # надо 3e6
        recombination_rate=1e-9, # надо 1e-8
        random_seed=seed,
        # record_migrations=True,    
        # record_provenance=True
    )
samples_p1 = treeseq.samples(population_id=0)
samples_p2 = treeseq.samples(population_id=1)
samples_p3 = treeseq.samples(population_id=3)

w1_1, w2_1 = mean_weight_proportions(treeseq, samples_p1)  
w1_2, w2_2 = mean_weight_proportions(treeseq, samples_p2)
w1_3, w2_3 = mean_weight_proportions(treeseq, samples_p3)

In [89]:
w1_1, w2_1, w1_2, w2_2, w1_3, w2_3

(np.float64(1.2921456172222223),
 np.float64(0.014973331111111113),
 np.float64(0.02624804722222222),
 np.float64(1.9400994372222218),
 np.float64(0.6516231866666667),
 np.float64(0.06155312888888889))

In [91]:
l1 = w1_1/(w1_1 + w2_1)
l2 = w1_2/(w1_2 + w2_2)
l3 = w1_3/(w1_3 + w2_3)
(l3 - l2)/(l1 - l2)

np.float64(0.9232429147604424)

Проверяем другое значение $\alpha$

In [92]:
def count_statistics2(treeseq):
    samples_p1 = treeseq.samples(population_id=0)
    samples_p2 = treeseq.samples(population_id=1)
    samples_p3 = treeseq.samples(population_id=3)

    w1_1, w2_1 = mean_weight_proportions(treeseq, samples_p1)  
    w1_2, w2_2 = mean_weight_proportions(treeseq, samples_p2)
    w1_3, w2_3 = mean_weight_proportions(treeseq, samples_p3)
    l1 = w1_1/(w1_1 + w2_1)
    l2 = w1_2/(w1_2 + w2_2)
    l3 = w1_3/(w1_3 + w2_3)
    return (l3 - l2)/(l1 - l2)

In [93]:
split = 0.4
dem = msprime.Demography() # TODO: key of migrations
dem.add_population(name="p1", initial_size=dem_size * 0.5)
dem.add_population(name="p2", initial_size=dem_size*0.5)
dem.add_population(name="p0", initial_size=dem_size)
dem.add_population(name="p3", initial_size=dem_size/2)
dem.add_admixture(time=T_p3, derived="p3", ancestral=["p1", "p2"], proportions=[split, 1-split])
dem.add_population_split(time=T_split, derived=["p1", "p2"], ancestral="p0")

treeseq = msprime.sim_ancestry(
    samples={"p1": p_size, "p2": p_size, "p3": p3_size},
    demography=dem,
    sequence_length=3e5, # надо 3e6
    recombination_rate=1e-9, # надо 1e-8
    random_seed=seed,
    # record_migrations=True,    
    # record_provenance=True
)
count_statistics2(treeseq)

np.float64(0.437605392147067)

Внимание, мы получаем что-то близкое!

### давайте w3 учитывать тоже для точности

In [94]:
def sample_neighbours_weights(treeseq, sample):
    occur1, occur2, occur3 = 0, 0, 0
    for tree in treeseq.trees():
        siblings = tree.siblings(sample)
        nearest = list([leaf for leaf in tree.leaves(sibling)] for sibling in siblings)[0]
        nearest_pop = [tree.population(nd) for nd in nearest]
        occur1 += nearest_pop.count(0) * tree.span #/ len(nearest_pop)
        occur2 += nearest_pop.count(1) * tree.span
        occur3 += nearest_pop.count(3) * tree.span
    return occur1, occur2

def mean_weight_proportions(ts, samples_set):
    proportions = np.array([sample_neighbours_weights(ts, s) for s in samples_set]) / ts.sequence_length
    return np.mean(proportions[:,0]), np.mean(proportions[:,1]), np.mean(proportions[:,2])
    
def guess_split_proportion2(split):
    dem = msprime.Demography() # TODO: key of migrations
    dem.add_population(name="p1", initial_size=dem_size * 0.5)
    dem.add_population(name="p2", initial_size=dem_size*0.5)
    dem.add_population(name="p0", initial_size=dem_size)
    dem.add_population(name="p3", initial_size=dem_size/2)
    dem.add_admixture(time=T_p3, derived="p3", ancestral=["p1", "p2"], proportions=[split, 1-split])
    dem.add_population_split(time=T_split, derived=["p1", "p2"], ancestral="p0")

    treeseq = msprime.sim_ancestry(
        samples={"p1": p_size, "p2": p_size, "p3": p3_size},
        demography=dem,
        sequence_length=3e5, # надо 3e6
        recombination_rate=1e-9, # надо 1e-8
        random_seed=seed,
        # record_migrations=True,    
        # record_provenance=True
    )

    samples_p1 = treeseq.samples(population_id=0)
    samples_p2 = treeseq.samples(population_id=1)
    samples_p3 = treeseq.samples(population_id=3)

    w1_1, w2_1, w3_1 = mean_weight_proportions(treeseq, samples_p1)  
    w1_2, w2_2, w3_2 = mean_weight_proportions(treeseq, samples_p2)
    w1_3, w2_3, w3_3 = mean_weight_proportions(treeseq, samples_p3)
    print(w1_1, w2_1, w3_1, w1_2, w2_2, w3_2, w1_3, w2_3, w3_3)
    l1 = w1_1/(w1_1 + w2_1 + w3_1)
    l2 = w1_2/(w1_2 + w2_2 + w3_2)
    l3 = w1_3/(w1_3 + w2_3 + w3_3)
    return (l3 - l2)/(l1 - l2)

In [95]:
a1 = guess_split_proportion2(0.65)
print(a1)

IndexError: index 2 is out of bounds for axis 1 with size 2

## Распределение через предков p3 (iconic)

Согласно здравому смыслу, alpha можно вычислить, если для каждого экземпляра из P3 посмотреть, восходит ли он к P1 или к P2 

In [77]:
treeseq = msprime.sim_ancestry(
        samples={"p1": p_size, "p2": p_size, "p3": p3_size},
        demography=dem,
        sequence_length=3e5, # надо 3e6
        recombination_rate=1e-9, # надо 1e-8
        random_seed=seed,
        # record_migrations=True,    
        # record_provenance=True
    )
samples_p3 = treeseq.samples(population_id=3)
alpha = 0
for s in samples_p3:
    des = 0
    for tree in treeseq.trees():
        p = tree.parent(s)
        while tree.population(p) == 3:
            # print(p)
            p = tree.parent(p)
        # id_p1 = 0, id_p2 = 1
        # то есть дробное число получившееся в конечном счете будет процентным содержанием p2
        # если сразу в p0 попадаем, то считаем это как P2, потому что всё равно потом берем 1-alpha
        des += tree.span * min(tree.population(p), 1)
    des = np.round(des/ treeseq.sequence_length)
    alpha += des
alpha /= (p3_size * 2)

In [78]:
1 - alpha

np.float64(0.9468333333333333)

Было $\alpha = 0.95$

"Вау, это работает"

TODO

r- reference panel???

Пусть изначально по k=100 экземпляров P1 и P2

k_learning = 25, k_validation=5

Чтобы не генерить кучу деревьев, мы в момент t=15 генерим n популяций c_i, где сплит c_i-1 популяции это 0.05*i (пусть для начала их будет, например, 7) , тогда у p1, p2 по (k_r + k_l) экземпляров, по k_v экземпляров у смешенных популяций

reference panel: по 100 (первых?) образцов из p1 и p2 . Фиксируем. 

Что мы берем. Рассматриваем подмножество 
берем по 100 референсов и по 25 лернигов
for el in Learning:
    строим distribution этой выборки референсов + el    
subgraph из референсов + один элемент из learning/validation. Два лернинга ничего друг про друга не знают!! И ВОТ ДЛЯ ЭТОГО ПОДГРАФА МЫ СЧИТАЕМ ДОЛЮ ВХОЖДЕНИЯ РЕФЕРЕНСНЫХ ЭЛЕМЕНТОВ среди соседей
Смотрим распределение для A и B, контролируя размер ветки:
- пусть s - кол-во соседей (через ближайшего предка) для конкретного экземпляра, и мы нормируем на это число распределение соседей из P1 и P2

Имея distrA, distrB (среднее mean{distr(el) for el in Learning}), мы оцениваем alpha для валидационных выборок по новым смешанным популяциям

аргмин по alpha - финальная цель (минимизировать ошибку по 2-норме)

Вектор референсов 

Цель: изучение генетической структуры и истории популяции при помощи ARG

Задачи:
Глобальная задача: 1) мы не знаем коэффициент перемешивания двух хорошо определенных популяций,для некоторой популяции между ними хотим определить коэффициент перемешивания. (не знаем распределение времен, знаем топологические свойства и несбалансированность генеологии)


ПОСЛЕ ТОГО КАК ПОЛУЧИМ ОЦЕНКИ на валидации
- точность оценки усреднения по нескольким образцам (bootstrap jackknife)

Начать читать recombinatorics!!